In [51]:
#pip install fairlearn
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model, decomposition, datasets
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
!pip install fairlearn
from fairlearn.metrics import demographic_parity_difference
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from fairlearn.reductions import GridSearch, EqualizedOdds

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
train = pd.read_csv("recid.csv")
num_cols = train.shape[1]
print("Number of columns in the dataset are : ",num_cols)
train.head(5)

Number of columns in the dataset are :  53


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,8/14/2013,Male,4/18/1947,69,Greater than 45,Other,...,1,Low,8/14/2013,7/7/2014,7/14/2014,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,1/27/2013,Male,1/22/1982,34,25 - 45,African-American,...,1,Low,1/27/2013,1/26/2013,2/5/2013,0,9,159,1,1
2,4,ed philo,ed,philo,4/14/2013,Male,5/14/1991,24,Less than 25,African-American,...,3,Low,4/14/2013,6/16/2013,6/16/2013,4,0,63,0,1
3,5,marcu brown,marcu,brown,1/13/2013,Male,1/21/1993,23,Less than 25,African-American,...,6,Medium,1/13/2013,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,3/26/2013,Male,1/22/1973,43,25 - 45,Other,...,1,Low,3/26/2013,NaN,NaN,2,0,1102,0,0


In [53]:
value_counts = train['race'].value_counts()
print("The different types of races being considered are :")
print(value_counts)

The different types of races being considered are :
African-American    3696
Caucasian           2454
Hispanic             637
Other                377
Asian                 32
Native American       18
Name: race, dtype: int64


In [54]:
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum()

days_b_screening_arrest     307
c_jail_in                   307
c_jail_out                  307
c_case_number                22
c_offense_date             1159
c_arrest_date              6077
c_days_from_compas           22
c_charge_desc                29
r_case_number              3743
r_charge_degree            3743
r_days_from_arrest         4898
r_offense_date             3743
r_charge_desc              3801
r_jail_in                  4898
r_jail_out                 4898
violent_recid              7214
vr_case_number             6395
vr_charge_degree           6395
vr_offense_date            6395
vr_charge_desc             6395
in_custody                  236
out_custody                 236
dtype: int64

In [55]:
train_new = train.drop(['id','name','first','last','compas_screening_date','dob','c_offense_date','c_arrest_date','r_case_number','r_charge_degree',
                'r_days_from_arrest','r_offense_date','r_charge_desc','r_jail_in','r_jail_out','violent_recid','start','end','vr_case_number',
                'vr_charge_degree','v_type_of_assessment','vr_offense_date','vr_charge_desc','screening_date','days_b_screening_arrest',
                'c_jail_in','c_jail_out','c_case_number','c_days_from_compas','v_screening_date','c_charge_desc','in_custody','out_custody',
                'type_of_assessment','is_recid'],axis=1)
num_cols = train_new.shape[1]
print("Number of columns in the dataset after removing unwanted columns are : ",num_cols)
train_new.head(5)

Number of columns in the dataset after removing unwanted columns are :  18


,sex,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,c_charge_degree,is_violent_recid,decile_score.1,score_text,v_decile_score,v_score_text,priors_count.1,event,two_year_recid
0,Male,69,Greater than 45,Other,0,1,0,0,0,F,0,1,Low,1,Low,0,0,0
1,Male,34,25 - 45,African-American,0,3,0,0,0,F,1,3,Low,1,Low,0,1,1
2,Male,24,Less than 25,African-American,0,4,0,1,4,F,0,4,Low,3,Low,4,0,1
3,Male,23,Less than 25,African-American,0,8,1,0,1,F,0,8,High,6,Medium,1,0,0
4,Male,43,25 - 45,Other,0,1,0,0,2,F,0,1,Low,1,Low,2,0,0


In [56]:
train_final = pd.get_dummies(train_new)
train_final.columns

Index(['age', 'juv_fel_count', 'decile_score', 'juv_misd_count',
       'juv_other_count', 'priors_count', 'is_violent_recid', 'decile_score.1',
       'v_decile_score', 'priors_count.1', 'event', 'two_year_recid',
       'sex_Female', 'sex_Male', 'age_cat_25 - 45', 'age_cat_Greater than 45',
       'age_cat_Less than 25', 'race_African-American', 'race_Asian',
       'race_Caucasian', 'race_Hispanic', 'race_Native American', 'race_Other',
       'c_charge_degree_F', 'c_charge_degree_M', 'score_text_High',
       'score_text_Low', 'score_text_Medium', 'v_score_text_High',
       'v_score_text_Low', 'v_score_text_Medium'],
      dtype='object')

In [57]:
num_cols = train_final.shape[1]
print("Number of columns after numeric conversion : ",num_cols)

Number of columns after numeric conversion :  31


In [58]:
train_final = train_final.drop('two_year_recid',axis=1)
X = train_final
Y = train['two_year_recid']

In [59]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.33,random_state=18)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (4833, 30)
Shape of X_test: (2381, 30)
Shape of y_train: (4833,)
Shape of y_test: (2381,)


In [60]:
# create a logistic regression object with iterations parameter and gini index parameter
logreg = LogisticRegression(max_iter=1000, solver='lbfgs', penalty='l2', random_state=18, n_jobs=-1, class_weight='balanced')

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)

print(classification_report(y_test,y_pred))

scores = cross_val_score(logreg,X_train,y_train,cv=5)
np.mean(scores)

""" Predicted +ve and are categorized as African American by the race variable"""
X_test[y_pred == 1]['race_African-American'].value_counts()[1]

""" Predicted +ve and are categorized as Caucasian by the race variable"""
X_test[y_pred == 1]['race_Caucasian'].value_counts()[1]

[[tn , fp],[fn , tp]] = confusion_matrix(y_test,y_pred)
print("True Negatives       : ", tn)
print("False Positives      : ", fp)
print("False Negatives      : ", fn)
print("True Positives       : ", tp)

# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of false positive predictions
fp_indices = np.where((y_pred == 1) & (y_test == 0))[0]

# filter test set to get only false positive predictions
fp_data = X_test.iloc[fp_indices]

# count number of false positives for African-American and Caucasian races
fp_African = fp_data['race_African-American'].sum()
fp_Caucasian = fp_data['race_Caucasian'].sum()

# calculate total number of false positives
fp_total = len(fp_data)

# print results
print("False positive predictions by the model")
print("African-American: ", fp_African)
print("Caucasian: ", fp_Caucasian)
print("Total False Positives: ", fp_total)

print("% of False positives")
print("African-American: %.2f" % (fp_African / fp_total))
print("Caucasian: %.2f" % (fp_Caucasian / fp_total))


# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of true positive predictions
tp_indices = np.where((y_pred == 1) & (y_test == 1))[0]

# filter test set to get only true positive predictions
tp_data = X_test.iloc[tp_indices]

# count number of true positives for African-American and Caucasian races
tp_African = tp_data['race_African-American'].sum()
tp_Caucasian = tp_data['race_Caucasian'].sum()

# calculate total number of true positives
tp_total = len(tp_data)

# print results
print("True positive predictions by the model")
print("African-American: ", tp_African)
print("Caucasian: ", tp_Caucasian)
print("Total True Positives: ", tp_total)

print("% of True positives")
print("African-American: %.2f" % (tp_African / tp_total))
print("Caucasian: %.2f" % (tp_Caucasian / tp_total))

Accuracy: 0.9042419151616967
                    predicted_recividated  predicted_not_recividated
is_recividated                        912                        157
is_not_recividated                     71                       1241
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1312
           1       0.93      0.85      0.89      1069

    accuracy                           0.90      2381
   macro avg       0.91      0.90      0.90      2381
weighted avg       0.91      0.90      0.90      2381

True Negatives       :  1241
False Positives      :  71
False Negatives      :  157
True Positives       :  912
False positive predictions by the model
African-American:  39
Caucasian:  20
Total False Positives:  71
% of False positives
African-American: 0.55
Caucasian: 0.28
True positive predictions by the model
African-American:  555
Caucasian:  269
Total True Positives:  912
% of True positives
African-American: 0.61
Caucasian: 0.

In [61]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# create a DecisionTreeClassifier object with Gini index as the splitting criterion
dt = DecisionTreeClassifier(criterion='gini')

# define the range of values to search for the best max_depth
param_grid = {'max_depth': range(1, 11)}

# create a GridSearchCV object with 5-fold cross-validation
grid = GridSearchCV(dt, param_grid, cv=5)

# fit the model to the training data
grid.fit(X_train, y_train)

# get the best value for max_depth
best_max_depth = grid.best_params_['max_depth']
print(best_max_depth)

2


In [62]:
# create a logistic regression object with iterations parameter and gini index parameter
from sklearn.tree import DecisionTreeClassifier

# create a DecisionTreeClassifier object with Gini index as the splitting criterion
dt = DecisionTreeClassifier(max_depth=1, criterion='gini')

# fit the model to the training data
dt.fit(X_train, y_train)

# make predictions on the test data
y_pred = dt.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)

print(classification_report(y_test,y_pred))

scores = cross_val_score(logreg,X_train,y_train,cv=5)
np.mean(scores)

""" Predicted +ve and are categorized as African American by the race variable"""
X_test[y_pred == 1]['race_African-American'].value_counts()[1]

""" Predicted +ve and are categorized as Caucasian by the race variable"""
X_test[y_pred == 1]['race_Caucasian'].value_counts()[1]

[[tn , fp],[fn , tp]] = confusion_matrix(y_test,y_pred)
print("True Negatives       : ", tn)
print("False Positives      : ", fp)
print("False Negatives      : ", fn)
print("True Positives       : ", tp)

# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of false positive predictions
fp_indices = np.where((y_pred == 1) & (y_test == 0))[0]

# filter test set to get only false positive predictions
fp_data = X_test.iloc[fp_indices]

# count number of false positives for African-American and Caucasian races
fp_African = fp_data['race_African-American'].sum()
fp_Caucasian = fp_data['race_Caucasian'].sum()

# calculate total number of false positives
fp_total = len(fp_data)

# print results
print("False positive predictions by the model")
print("African-American: ", fp_African)
print("Caucasian: ", fp_Caucasian)
print("Total False Positives: ", fp_total)

print("% of False positives")
print("African-American: %.2f" % (fp_African / fp_total))
print("Caucasian: %.2f" % (fp_Caucasian / fp_total))


# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of true positive predictions
tp_indices = np.where((y_pred == 1) & (y_test == 1))[0]

# filter test set to get only true positive predictions
tp_data = X_test.iloc[tp_indices]

# count number of true positives for African-American and Caucasian races
tp_African = tp_data['race_African-American'].sum()
tp_Caucasian = tp_data['race_Caucasian'].sum()

# calculate total number of true positives
tp_total = len(tp_data)

# print results
print("True positive predictions by the model")
print("African-American: ", tp_African)
print("Caucasian: ", tp_Caucasian)
print("Total True Positives: ", tp_total)

print("% of True positives")
print("African-American: %.2f" % (tp_African / tp_total))
print("Caucasian: %.2f" % (tp_Caucasian / tp_total))

Accuracy: 0.8903821923561529
                    predicted_recividated  predicted_not_recividated
is_recividated                        856                        213
is_not_recividated                     48                       1264
              precision    recall  f1-score   support

           0       0.86      0.96      0.91      1312
           1       0.95      0.80      0.87      1069

    accuracy                           0.89      2381
   macro avg       0.90      0.88      0.89      2381
weighted avg       0.90      0.89      0.89      2381

True Negatives       :  1264
False Positives      :  48
False Negatives      :  213
True Positives       :  856
False positive predictions by the model
African-American:  26
Caucasian:  14
Total False Positives:  48
% of False positives
African-American: 0.54
Caucasian: 0.29
True positive predictions by the model
African-American:  518
Caucasian:  252
Total True Positives:  856
% of True positives
African-American: 0.61
Caucasian: 0.

In [63]:
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)

print(classification_report(y_test,y_pred))

scores = cross_val_score(logreg,X_train,y_train,cv=5)
np.mean(scores)

""" Predicted +ve and are categorized as African American by the race variable"""
X_test[y_pred == 1]['race_African-American'].value_counts()[1]

""" Predicted +ve and are categorized as Caucasian by the race variable"""
X_test[y_pred == 1]['race_Caucasian'].value_counts()[1]

[[tn , fp],[fn , tp]] = confusion_matrix(y_test,y_pred)
print("True Negatives       : ", tn)
print("False Positives      : ", fp)
print("False Negatives      : ", fn)
print("True Positives       : ", tp)

# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of false positive predictions
fp_indices = np.where((y_pred == 1) & (y_test == 0))[0]

# filter test set to get only false positive predictions
fp_data = X_test.iloc[fp_indices]

# count number of false positives for African-American and Caucasian races
fp_African = fp_data['race_African-American'].sum()
fp_Caucasian = fp_data['race_Caucasian'].sum()

# calculate total number of false positives
fp_total = len(fp_data)

# print results
print("False positive predictions by the model")
print("African-American: ", fp_African)
print("Caucasian: ", fp_Caucasian)
print("Total False Positives: ", fp_total)

print("% of False positives")
print("African-American: %.2f" % (fp_African / fp_total))
print("Caucasian: %.2f" % (fp_Caucasian / fp_total))


# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of true positive predictions
tp_indices = np.where((y_pred == 1) & (y_test == 1))[0]

# filter test set to get only true positive predictions
tp_data = X_test.iloc[tp_indices]

# count number of true positives for African-American and Caucasian races
tp_African = tp_data['race_African-American'].sum()
tp_Caucasian = tp_data['race_Caucasian'].sum()

# calculate total number of true positives
tp_total = len(tp_data)

# print results
print("True positive predictions by the model")
print("African-American: ", tp_African)
print("Caucasian: ", tp_Caucasian)
print("Total True Positives: ", tp_total)

print("% of True positives")
print("African-American: %.2f" % (tp_African / tp_total))
print("Caucasian: %.2f" % (tp_Caucasian / tp_total))

Accuracy: 0.9004619907601848
                    predicted_recividated  predicted_not_recividated
is_recividated                        931                        138
is_not_recividated                     99                       1213
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1312
           1       0.90      0.87      0.89      1069

    accuracy                           0.90      2381
   macro avg       0.90      0.90      0.90      2381
weighted avg       0.90      0.90      0.90      2381

True Negatives       :  1213
False Positives      :  99
False Negatives      :  138
True Positives       :  931
False positive predictions by the model
African-American:  56
Caucasian:  24
Total False Positives:  99
% of False positives
African-American: 0.57
Caucasian: 0.24
True positive predictions by the model
African-American:  569
Caucasian:  272
Total True Positives:  931
% of True positives
African-American: 0.61
Caucasian: 0.

In [64]:
base_estimator = DecisionTreeClassifier(criterion='gini')
adaboost = AdaBoostClassifier(estimator=base_estimator, n_estimators=200)
adaboost.fit(X_train, y_train)

# make predictions on the test data
y_pred = adaboost.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)

print(classification_report(y_test,y_pred))

scores = cross_val_score(logreg,X_train,y_train,cv=5)
np.mean(scores)

""" Predicted +ve and are categorized as African American by the race variable"""
X_test[y_pred == 1]['race_African-American'].value_counts()[1]

""" Predicted +ve and are categorized as Caucasian by the race variable"""
X_test[y_pred == 1]['race_Caucasian'].value_counts()[1]

[[tn , fp],[fn , tp]] = confusion_matrix(y_test,y_pred)
print("True Negatives       : ", tn)
print("False Positives      : ", fp)
print("False Negatives      : ", fn)
print("True Positives       : ", tp)

# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of false positive predictions
fp_indices = np.where((y_pred == 1) & (y_test == 0))[0]

# filter test set to get only false positive predictions
fp_data = X_test.iloc[fp_indices]

# count number of false positives for African-American and Caucasian races
fp_African = fp_data['race_African-American'].sum()
fp_Caucasian = fp_data['race_Caucasian'].sum()

# calculate total number of false positives
fp_total = len(fp_data)

# print results
print("False positive predictions by the model")
print("African-American: ", fp_African)
print("Caucasian: ", fp_Caucasian)
print("Total False Positives: ", fp_total)

print("% of False positives")
print("African-American: %.2f" % (fp_African / fp_total))
print("Caucasian: %.2f" % (fp_Caucasian / fp_total))


# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of true positive predictions
tp_indices = np.where((y_pred == 1) & (y_test == 1))[0]

# filter test set to get only true positive predictions
tp_data = X_test.iloc[tp_indices]

# count number of true positives for African-American and Caucasian races
tp_African = tp_data['race_African-American'].sum()
tp_Caucasian = tp_data['race_Caucasian'].sum()

# calculate total number of true positives
tp_total = len(tp_data)

# print results
print("True positive predictions by the model")
print("African-American: ", tp_African)
print("Caucasian: ", tp_Caucasian)
print("Total True Positives: ", tp_total)

print("% of True positives")
print("African-American: %.2f" % (tp_African / tp_total))
print("Caucasian: %.2f" % (tp_Caucasian / tp_total))

Accuracy: 0.8500629987400252
                    predicted_recividated  predicted_not_recividated
is_recividated                        882                        187
is_not_recividated                    170                       1142
              precision    recall  f1-score   support

           0       0.86      0.87      0.86      1312
           1       0.84      0.83      0.83      1069

    accuracy                           0.85      2381
   macro avg       0.85      0.85      0.85      2381
weighted avg       0.85      0.85      0.85      2381

True Negatives       :  1142
False Positives      :  170
False Negatives      :  187
True Positives       :  882
False positive predictions by the model
African-American:  86
Caucasian:  49
Total False Positives:  170
% of False positives
African-American: 0.51
Caucasian: 0.29
True positive predictions by the model
African-American:  542
Caucasian:  257
Total True Positives:  882
% of True positives
African-American: 0.61
Caucasian: 

In [65]:
from fairlearn.metrics import equalized_odds_difference

# Define the sensitive features
sensitive_features = np.array([X_train['race_Caucasian'], X_train['race_African-American']]).T
sensitive_features_test = np.array([X_test['race_Caucasian'], X_test['race_African-American']]).T

# Define the fairness constraint
constraint = EqualizedOdds()

# Define the model and the optimizer
model = RandomForestClassifier(n_estimators=100)
optimizer = ExponentiatedGradient(model, constraints=constraint)

# Fit the optimizer to the training data
optimizer.fit(X_train, y_train, sensitive_features=sensitive_features)

# Evaluate the fairness of the model on the test data
y_pred = optimizer.predict(X_test)
cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)
dp_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=sensitive_features_test)
eo_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=sensitive_features_test)
print("Demographic parity difference: ", dp_diff)
print("Equalized odds difference: ", eo_diff)
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix for the Caucasian group
cm_caucasian = confusion_matrix(y_test[sensitive_features_test[:, 0] == 1], y_pred[sensitive_features_test[:, 0] == 1])
false_positives_caucasian = cm_caucasian[0, 1]

# Compute the confusion matrix for the African-American group
cm_african_american = confusion_matrix(y_test[sensitive_features_test[:, 1] == 1], y_pred[sensitive_features_test[:, 1] == 1])
false_positives_african_american = cm_african_american[0, 1]

print("False positives for the Caucasian group: ", false_positives_caucasian)
print("False positives for the African-American group: ", false_positives_african_american)

                    predicted_recividated  predicted_not_recividated
is_recividated                        933                        136
is_not_recividated                    103                       1209
Demographic parity difference:  0.18905859271637632
Equalized odds difference:  0.04932283245536257
False positives for the Caucasian group:  25
False positives for the African-American group:  58


In [69]:
# Compute the confusion matrix for the Caucasian group
cm_caucasian = confusion_matrix(y_test[sensitive_features_test[:, 0] == 1], y_pred[sensitive_features_test[:, 0] == 1])
true_positives_caucasian = cm_caucasian[1, 1]
false_positives_caucasian = cm_caucasian[0, 1]

# Compute the confusion matrix for the African-American group
cm_african_american = confusion_matrix(y_test[sensitive_features_test[:, 1] == 1], y_pred[sensitive_features_test[:, 1] == 1])
true_positives_african_american = cm_african_american[1, 1]
false_positives_african_american = cm_african_american[0, 1]

# Calculate probabilities
prob_recidivates_given_predicted_positive_caucasian = true_positives_caucasian / (true_positives_caucasian + false_positives_caucasian)
prob_recidivates_given_predicted_positive_african_american = true_positives_african_american / (true_positives_african_american + false_positives_african_american)

print("Probability [individual recidivates | predicted +ve, African American]: ", prob_recidivates_given_predicted_positive_african_american)
print("Probability [individual recidivates | predicted +ve, Caucasian]: ", prob_recidivates_given_predicted_positive_caucasian)


Probability [individual recidivates | predicted +ve, African American]:  0.9074960127591707
Probability [individual recidivates | predicted +ve, Caucasian]:  0.9161073825503355
